# Comenzamos obteniendo los datos

#### Cargamos paquetes y datasets

In [ ]:
# !pip install scipy

In [2]:
import pandas as pd
import numpy as np
import pyarrow
from scipy.sparse import hstack, csr_matrix
from scipy.sparse import vstack
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import sklearn

# Instalar lightfm con pip
!pip install lightfm

# Importar lightfm
from lightfm import LightFM

import requests
from bs4 import BeautifulSoup

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808333 sha256=f7d6ef9acc105aa54011f36f75b8d66abc7dfebccf89d8ffdf8458e52dc015c0
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
pd.set_option('display.max_columns', 100)    # Muestra hasta 10 columnas
# pd.set_option('display.max_rows', 100)      # Muestra hasta 100 filas

In [42]:
# df_anime_base = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/anime.csv") #csv BASE
# df_anime = pd.read_pickle ("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/df_anime_editado.pickle") #df EDITADO

# df_anime_sinopsis = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/anime_with_synopsis.csv")

# df_anime_list = pd.read_csv ("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/animelist.csv")

# df_rating_complete = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/rating_complete.csv")

# df_watching_status = pd.read_csv ("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/watching_status.csv")

ParserError: ignored

#### Descripcion de datasets de origen

**df_animelist = animelist.csv** have the list of all animes register by the user with the respective score, watching status and numbers of episodes watched. This dataset contains 109 Million row, 17.562 different animes and 325.772 different users. The file have the following columns:

- user_id: non identifiable randomly generated user id.

- anime_id: MyAnemlist ID of the anime. (e.g. 1).

- score: score between 1 to 10 given by the user. 0 if the user didn't assign a score. (e.g. 10)

- watching_status: state ID from this anime in the anime list of this user. (e.g. 2)

- watched_episodes: numbers of episodes watched by the user. (e.g. 24)

In [ ]:
df_anime_list

**df_anime_sinopsis = anime_sinopsis.csv**

In [ ]:
df_anime_sinopsis

---

**df_watching_status = watching_status.csv** describe every possible status of the column: "watching_status" in animelist.csv.

In [ ]:
df_watching_status

---
**df_rating_complete = rating_complete.csv** is a subset of animelist.csv. This dataset only considers animes that the user has watched completely (watching_status==2) and gave it a score (score!=0). This dataset contains 57 Million ratings applied to 16.872 animes by 310.059 users. This file have the following columns:

- user_id: non identifiable randomly generated user id.

- anime_id: - MyAnimelist ID of the anime that this user has rated.

- rating: rating that this user has assigned.

In [ ]:
df_rating_complete

---
**df_anime = anime.csv** contain general information of every anime (17.562 different anime) like genre, stats, studio, etc. This file have the following columns:

- MAL_ID: MyAnimelist ID of the anime. (e.g. 1)

- Name: full name of the anime. (e.g. Cowboy Bebop)

- Score: average score of the anime given from all users in MyAnimelist database. (e.g. 8.78)

- Genres: comma separated list of genres for this anime. (e.g. Action, Adventure, Comedy, Drama, Sci-Fi, Space)

- English name: full name in english of the anime. (e.g. Cowboy Bebop)

- Japanese name: full name in japanses of the anime. (e.g. カウボーイビバップ)

- Type: TV, movie, OVA, etc. (e.g. TV)

- Episodes': number of chapters. (e.g. 26)

- Aired: broadcast date. (e.g. Apr 3, 1998 to Apr 24, 1999)

- Premiered: season premiere. (e.g. Spring 1998)

- Producers: comma separated list of produducers (e.g. Bandai Visual)

- Licensors: comma separated list of licensors (e.g. Funimation, Bandai Entertainment)

- Studios: comma separated list of studios (e.g. Sunrise)

- Source: Manga, Light novel, Book, etc. (e.g Original)

- Duration: duration of the anime per episode (e.g 24 min. per ep.)

- Rating: age rate (e.g. R - 17+ (violence & profanity))

- Ranked: position based in the score. (e.g 28)

- Popularity: position based in the the number of users who have added the anime to their list. (e.g 39)

- Members: number of community members that are in this anime's "group". (e.g. 1251960)

- Favorites: number of users who have the anime as "favorites". (e.g. 61,971)

- Watching: number of users who are watching the anime. (e.g. 105808)

- Completed: number of users who have complete the anime. (e.g. 718161)

- On-Hold: number of users who have the anime on Hold. (e.g. 71513)

- Dropped: number of users who have dropped the anime. (e.g. 26678)

- Plan to Watch': number of users who plan to watch the anime. (e.g. 329800)

- Score-10': number of users who scored 10. (e.g. 229170)

- Score-9': number of users who scored 9. (e.g. 182126)

- Score-8': number of users who scored 8. (e.g. 131625)

- Score-7': number of users who scored 7. (e.g. 62330)

- Score-6': number of users who scored 6. (e.g. 20688)

- Score-5': number of users who scored 5. (e.g. 8904)

- Score-4': number of users who scored 4. (e.g. 3184)

- Score-3': number of users who scored 3. (e.g. 1357)

- Score-2': number of users who scored 2. (e.g. 741)

- Score-1': number of users who scored 1. (e.g. 1580)

In [ ]:
df_anime

### Exploramos los datasets

In [ ]:
df_anime_list[df_anime_list["anime_id"]==1]

In [ ]:
df_rating_complete[df_rating_complete["anime_id"]==1]# ["rating"].value_counts()

In [ ]:
df_anime_sinopsis[df_anime_sinopsis["MAL_ID"]==1]

In [ ]:
df_anime[df_anime["MAL_ID"]==1]

# Editando df_anime

#### Analizamos los datos del df

In [ ]:
# vemos las columnas que componen el df
df_anime.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [ ]:
df_anime.head(2)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0


In [ ]:
# creamos un subset con las columnas que nos interesan

df_anime = df_anime[['MAL_ID',
                     'Score',
                     'Genres',
                     'English name',
                     'Japanese name',
                     'Type',
                     'Episodes',
                     'Aired',
                     'Premiered',
                     'Producers',
                     'Licensors',
                     'Studios',
                     'Source',
                     'Duration',
                     'Rating',
                     # 'Ranked', 'Popularity',
                     # 'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
                     # 'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
                     # 'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'
                     ]]


In [ ]:
print(df_anime.shape)
df_anime.head(2)

(17562, 15)


,MAL_ID,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating
0,1,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity)
1,5,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity)


In [ ]:
# Analizamos el porcentaje de datos desconocidos del df
round(df_anime.eq('Unknown').sum()/len(df_anime)*100,2)

MAL_ID            0.00
Score            29.27
Genres            0.36
English name     60.16
Japanese name     0.27
Type              0.21
Episodes          2.94
Aired             1.76
Premiered        72.98
Producers        44.38
Licensors        77.53
Studios          40.31
Source           20.31
Duration          3.16
Rating            3.92
dtype: float64

#### Trabajamos la columna "Duration" y agrupamos cada 10 minutos

In [ ]:
df_anime["Duration"].value_counts()

24 min. per ep.          1723
23 min. per ep.          1070
25 min. per ep.           846
30 min. per ep.           746
2 min.                    745
                         ... 
1 hr. 52 min. per ep.       1
2 hr. 33 min.               1
2 hr. 21 min.               1
19 sec.                     1
55 sec. per ep.             1
Name: Duration, Length: 313, dtype: int64

In [ ]:
horas = df_anime["Duration"].str.extract("(\d+) hr\.").fillna(0).astype(int) # extraemos las horas de la duracion
minutos = df_anime["Duration"].str.extract("(\d+) min\.").fillna(0).astype(int) # extraemos los minutos de la duracion
segundos = df_anime["Duration"].str.extract("(\d+) sec\.").fillna(0).astype(int) # extraemos los segundos de la duracion

# redondeamos todos los que duran segundos, a 1 minuto
segundos[segundos != 0] = 1

df_anime["Duration_num"] = horas * 60 + minutos + segundos

In [ ]:
# df_anime[df_anime["Duration_num"]<10]["Duration_num"].value_counts()
df_anime["Duration_num"].max()

167

In [ ]:
# Agrupamos las duraciones en grupos de cada 10 minutos
duration_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170]
duration_etiquetas = "0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100",  "100-110", "110-120", "120-130", "130-140", "140-150", "150-160", "160-170"

df_anime["Duration"] = pd.cut(df_anime["Duration_num"],
                              bins = duration_bins,
                              labels = duration_etiquetas,
                              right = False)

In [ ]:
df_anime["Duration"].value_counts()

0-10       5947
20-30      5890
10-20      1832
30-40      1297
40-50       582
90-100      422
50-60       388
80-90       308
70-80       266
60-70       205
100-110     201
110-120     125
120-130      52
130-140      28
140-150       8
150-160       6
160-170       5
Name: Duration, dtype: int64

#### Trabajamos la columna "Premiered", completando los desconocidos con valores de "Aired"

In [ ]:
df_anime["Aired"].value_counts()

Unknown                         309
2005                             57
2004                             49
2021 to ?                        46
2003                             46
                               ... 
Dec 18, 2009 to Jul 26, 2010      1
Dec 26, 2009                      1
Apr 5, 1993 to Jan 27, 1994       1
Jan 25, 1992                      1
Mar 17, 2021 to ?                 1
Name: Aired, Length: 11947, dtype: int64

In [ ]:
# extraemos el año de primera emision
year_aired = df_anime["Aired"].str.extract(r'(\b\d{4}\b)').fillna(0)
year_aired

,0
0,1998
1,2001
2,1998
3,2002
4,2004
...,...
17557,2021
17558,2021
17559,2021
17560,0


In [ ]:
df_anime["Premiered"]

0        Spring 1998
1            Unknown
2        Spring 1998
3        Summer 2002
4          Fall 2004
            ...     
17557        Unknown
17558        Unknown
17559    Summer 2021
17560        Unknown
17561    Summer 2021
Name: Premiered, Length: 17562, dtype: object

In [ ]:
# Extraemos el año de estreno, reemplazando los valores faltantes con el año de primera emision
df_anime["Premiered"] = df_anime["Premiered"].str.extract("(\d+)").fillna(year_aired).astype(int)

In [ ]:
df_anime["Premiered"].value_counts()

2017    926
2016    895
2018    881
2014    854
2015    790
       ... 
1953      2
1951      2
1937      2
1945      1
1944      1
Name: Premiered, Length: 102, dtype: int64

#### Trabajamos la columna "Rating" y guardamos archivo pickle

In [ ]:
df_anime[df_anime["Rating"]=="Unknown"].shape

(688, 16)

Hacemos Web Scrapping para extraer de la pagina MyAnimeList los ratings faltantes del df

In [ ]:
# Hallamos los ratings desconocidos en el df
unknown_rating = df_anime[df_anime["Rating"]=="Unknown"]

# Iteramos sobre los rating desconocidos del df
for index, row in unknown_rating.iterrows():
  anime_id = row ["MAL_ID"]

  busqueda_url = f'https://myanimelist.net/anime/{anime_id}'
  response = requests.get(busqueda_url)

  if response.status_code == 200:
    # Analizamos la pagina web
    soup = BeautifulSoup(response.content, "html.parser")

    # Hallamos Rating dentro de la pagina
    rating_unknown = soup.find("span", class_="dark_text", string="Rating:")

    # Extraemos el valor y actualizamos el df
    if rating_unknown:
      rating_text = rating_unknown.find_next_sibling(string=True).text.strip()
      if rating_text != "None":
        print(f"{anime_id} - {rating_text}")
        df_anime.at[index, "Rating"] = rating_text

714 - PG-13 - Teens 13 or older
1712 - PG-13 - Teens 13 or older
2382 - PG-13 - Teens 13 or older
2739 - R+ - Mild Nudity
2778 - PG-13 - Teens 13 or older
3222 - PG-13 - Teens 13 or older


KeyboardInterrupt: ignored

In [ ]:
df_anime[df_anime["Rating"]=="Unknown"].shape

(682, 16)

Si bien el proceso obtiene resultados, es lento y no representa un valor que posea significancia en el df, con lo cual procederemos a completar valores faltantes con la moda

In [ ]:
# Reemplazamos los valores desconocidos restantes con la moda: "PG-13 - Teens 13 or older"
moda = df_anime["Rating"].mode()[0]

df_anime.loc[df_anime["Rating"]=="Unknown", "Rating"] = moda
df_anime[df_anime["Rating"]=="Unknown"].shape

(0, 16)

In [ ]:
round(df_anime.eq('Unknown').sum()/len(df_anime)*100,2)

MAL_ID            0.00
Score            29.27
Genres            0.36
English name     60.16
Japanese name     0.27
Type              0.21
Episodes          2.94
Aired             1.76
Premiered         0.00
Producers        44.38
Licensors        77.53
Studios          40.31
Source           20.31
Duration          0.00
Rating            0.00
Duration_num      0.00
dtype: float64

In [ ]:
# df_anime[df_anime["Source"]=="Unknown"]
df_anime["Duration"].value_counts()

0-10       5947
20-30      5890
10-20      1832
30-40      1297
40-50       582
90-100      422
50-60       388
80-90       308
70-80       266
60-70       205
100-110     201
110-120     125
120-130      52
130-140      28
140-150       8
150-160       6
160-170       5
Name: Duration, dtype: int64

#### Agrupamos por cantidad de episodios

In [ ]:
df_anime["Episodes"].value_counts()

1      8381
12     1445
2      1368
13      769
3       631
       ... 
145       1
330       1
193       1
220       1
160       1
Name: Episodes, Length: 201, dtype: int64

In [ ]:
# Agrupamos la cantidad de episodios en grupos
episodios_bins = [0, 2, 5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 300, 500, 10000]
episodios_etiquetas = "1-2", "2-5", "5-10", "10-15", "15-20", "20-25", "25-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100", "100-150", "150-200", "200-300", "300-500", "500+"

# Crear una máscara booleana para identificar los valores conocidos
mask_known = df_anime["Episodes"] != "Unknown"

# Crear una nueva serie con los valores categóricos basados solo en los valores conocidos
df_anime["Episodes_Categorized"] = pd.cut(pd.to_numeric(df_anime.loc[mask_known, "Episodes"]),
                                          bins=episodios_bins,
                                          labels=episodios_etiquetas,
                                          right=False)

# Reemplazar los valores categorizados en el DataFrame original
df_anime.loc[mask_known, "Episodes"] = df_anime["Episodes_Categorized"]

# Eliminar la columna temporal si ya no es necesaria
df_anime.drop(columns="Episodes_Categorized", inplace=True)


In [ ]:
df_anime["Episodes"].value_counts()

1-2        8381
10-15      2580
2-5        2450
25-30       860
5-10        777
Unknown     516
50-60       510
20-25       446
30-40       272
40-50       261
15-20       131
100-150     114
60-70        64
70-80        59
150-200      41
200-300      24
90-100       22
80-90        22
300-500      16
500+         16
Name: Episodes, dtype: int64

In [ ]:
round(df_anime.eq('Unknown').sum()/len(df_anime)*100,2)

MAL_ID            0.00
Score            29.27
Genres            0.36
English name     60.16
Japanese name     0.27
Type              0.21
Episodes          2.94
Aired             1.76
Premiered         0.00
Producers        44.38
Licensors        77.53
Studios          40.31
Source           20.31
Duration          0.00
Rating            0.00
Duration_num      0.00
dtype: float64

In [ ]:
# creamos un subset filtrando las columnas que nos interesan para seguir trabajando

df_anime = df_anime[['MAL_ID',
                    #  'Score',
                     'Genres',
                    #  'English name',
                    #  'Japanese name',
                     'Type',
                     'Episodes',
                    #  'Aired',
                     'Premiered',
                     'Producers',
                    #  'Licensors',
                     'Studios',
                     'Source',
                     'Duration',
                     'Rating',
                    #  "Duration_num",
                     # 'Ranked', 'Popularity',
                     # 'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
                     # 'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
                     # 'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'
                     ]]


#### Explotamos columnas y creamos la matrix de animes, para hacer similaridades

In [ ]:
print(df_anime.shape)
df_anime.head(2)

(17562, 10)


,MAL_ID,Genres,Type,Episodes,Premiered,Producers,Studios,Source,Duration,Rating
0,1,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,25-30,1998,Bandai Visual,Sunrise,Original,20-30,R - 17+ (violence & profanity)
1,5,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1-2,2001,"Sunrise, Bandai Visual",Bones,Original,110-120,R - 17+ (violence & profanity)


In [ ]:
# Explotamos columnas
print(f"shape antes: {df_anime.shape}")

df_anime = df_anime.assign(Genres=df_anime["Genres"].str.split(", ")).explode("Genres")
df_anime = df_anime.assign(Producers=df_anime["Producers"].str.split(", ")).explode("Producers")
df_anime = df_anime.assign(Studios=df_anime["Studios"].str.split(", ")).explode("Studios")

df_anime.reset_index()

print(f"shape despues: {df_anime.shape}")
df_anime[df_anime["MAL_ID"]==5]

shape antes: (17562, 10)
shape despues: (96949, 10)


,MAL_ID,Genres,Type,Episodes,Premiered,Producers,Studios,Source,Duration,Rating
1,5,Action,Movie,1-2,2001,Sunrise,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Action,Movie,1-2,2001,Bandai Visual,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Drama,Movie,1-2,2001,Sunrise,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Drama,Movie,1-2,2001,Bandai Visual,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Mystery,Movie,1-2,2001,Sunrise,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Mystery,Movie,1-2,2001,Bandai Visual,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Sci-Fi,Movie,1-2,2001,Sunrise,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Sci-Fi,Movie,1-2,2001,Bandai Visual,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Space,Movie,1-2,2001,Sunrise,Bones,Original,110-120,R - 17+ (violence & profanity)
1,5,Space,Movie,1-2,2001,Bandai Visual,Bones,Original,110-120,R - 17+ (violence & profanity)


In [ ]:
df_anime["MAL_ID"] = df_anime["MAL_ID"].astype(str)
df_anime["Premiered"] = df_anime["Premiered"].astype(str)

df_anime.dtypes

MAL_ID         object
Genres         object
Type           object
Episodes       object
Premiered      object
Producers      object
Studios        object
Source         object
Duration     category
Rating         object
dtype: object

In [ ]:
# Instanciamos one hot encoder y obtenemos los nombres de las columnas
onehot = OneHotEncoder (handle_unknown="ignore", sparse_output = False)

# Hacemos fit_transform
onehot_df = onehot.fit_transform(df_anime.drop("MAL_ID", axis=1))
new_column_names = onehot.get_feature_names_out()

# # Creamos un nuevo DataFrame con las nuevas columnas y sus nombres
df_anime_matrix = pd.DataFrame(onehot_df, columns=list(new_column_names),)

In [ ]:
# Reseteamos el indice para mergear
df_anime = df_anime.reset_index(drop=True)

In [ ]:
# Agregamos ID al df matriz
df_anime_matrix ["MAL_ID"] = df_anime ["MAL_ID"]

In [ ]:
# Reordenamos las columnas, llevando ID al principio
column_order = list(df_anime_matrix.columns)
last_column = column_order[-1]  # Obtener la última columna
column_order = [last_column] + column_order[:-1]  # Mover la última columna al primer lugar

df_anime_matrix = df_anime_matrix[column_order]
df_anime_matrix

,MAL_ID,Genres_Action,Genres_Adventure,Genres_Cars,Genres_Comedy,Genres_Dementia,Genres_Demons,Genres_Drama,Genres_Ecchi,Genres_Fantasy,...,Duration_60-70,Duration_70-80,Duration_80-90,Duration_90-100,Rating_G - All Ages,Rating_PG - Children,Rating_PG-13 - Teens 13 or older,Rating_R - 17+ (violence & profanity),Rating_R+ - Mild Nudity,Rating_Rx - Hentai
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96944,48491,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
96945,48491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
96946,48491,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
96947,48492,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# Agrupamos por MAL_ID
df_anime_matrix = df_anime_matrix.groupby("MAL_ID").max().reset_index()

NameError: ignored

In [ ]:
# Cambiamos a INT la columna MAL_ID
df_anime_matrix["MAL_ID"] = df_anime_matrix["MAL_ID"].astype(int)

# Ordenamos de menor a mayor y seteamos la columna como indice
df_anime_matrix = df_anime_matrix.sort_values("MAL_ID").set_index("MAL_ID")

In [ ]:
df_anime_matrix

,Genres_Action,Genres_Adventure,Genres_Cars,Genres_Comedy,Genres_Dementia,Genres_Demons,Genres_Drama,Genres_Ecchi,Genres_Fantasy,Genres_Game,Genres_Harem,Genres_Hentai,Genres_Historical,Genres_Horror,Genres_Josei,Genres_Kids,Genres_Magic,Genres_Martial Arts,Genres_Mecha,Genres_Military,Genres_Music,Genres_Mystery,Genres_Parody,Genres_Police,Genres_Psychological,Genres_Romance,Genres_Samurai,Genres_School,Genres_Sci-Fi,Genres_Seinen,Genres_Shoujo,Genres_Shoujo Ai,Genres_Shounen,Genres_Shounen Ai,Genres_Slice of Life,Genres_Space,Genres_Sports,Genres_Super Power,Genres_Supernatural,Genres_Thriller,Genres_Unknown,Genres_Vampire,Genres_Yaoi,Genres_Yuri,Type_Movie,Type_Music,Type_ONA,Type_OVA,Type_Special,Type_TV,...,Studios_feel.,Studios_foodunited.,Studios_helo.inc,Studios_iDRAGONS Creative Studio,Studios_ixtl,Studios_l-a-unch・BOX,Studios_monofilmo,Studios_pH Studio,Studios_production doA,Studios_teamKG,Studios_ufotable,Source_4-koma manga,Source_Book,Source_Card game,Source_Digital manga,Source_Game,Source_Light novel,Source_Manga,Source_Music,Source_Novel,Source_Original,Source_Other,Source_Picture book,Source_Radio,Source_Unknown,Source_Visual novel,Source_Web manga,Duration_0-10,Duration_10-20,Duration_100-110,Duration_110-120,Duration_120-130,Duration_130-140,Duration_140-150,Duration_150-160,Duration_160-170,Duration_20-30,Duration_30-40,Duration_40-50,Duration_50-60,Duration_60-70,Duration_70-80,Duration_80-90,Duration_90-100,Rating_G - All Ages,Rating_PG - Children,Rating_PG-13 - Teens 13 or older,Rating_R - 17+ (violence & profanity),Rating_R+ - Mild Nudity,Rating_Rx - Hentai
MAL_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

### Trazamos la matriz de similitud coseno y obtenemos recomendaciones

In [ ]:
# Calculamos la similitud coseno
similitud = cosine_similarity(df_anime_matrix, df_anime_matrix)

# Pasamos a df con indice = MAL_ID
df_anime_similitud = pd.DataFrame(similitud, columns = df_anime_matrix.index, index = df_anime_matrix.index)

In [ ]:
df_anime_similitud

MAL_ID,1,5,6,7,8,15,16,17,18,19,...,48456,48466,48470,48471,48480,48481,48483,48488,48491,48492
MAL_ID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.500000,0.642857,0.518563,0.222375,0.276026,0.276026,0.231455,0.285714,0.194461,...,0.231455,0.253546,0.357143,0.241747,0.084515,0.080582,0.161165,0.142857,0.241747,0.253546
5,0.500000,1.000000,0.214286,0.324102,0.000000,0.069007,0.069007,0.000000,0.142857,0.129641,...,0.231455,0.169031,0.214286,0.161165,0.000000,0.080582,0.000000,0.142857,0.000000,0.084515
6,0.642857,0.214286,1.000000,0.453743,0.370625,0.483046,0.414039,0.462910,0.428571,0.324102,...,0.231455,0.253546,0.357143,0.241747,0.253546,0.161165,0.322329,0.071429,0.402911,0.253546
7,0.518563,0.324102,0.453743,1.000000,0.336336,0.313112,0.250490,0.280056,0.324102,0.294118,...,0.210042,0.306786,0.388922,0.146254,0.076696,0.219382,0.219382,0.194461,0.146254,0.306786
8,0.222375,0.000000,0.370625,0.336336,1.000000,0.358057,0.214834,0.400320,0.296500,0.269069,...,0.080064,0.087706,0.222375,0.083624,0.263117,0.167248,0.250873,0.148250,0.250873,0.175412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48481,0.080582,0.080582,0.161165,0.219382,0.167248,0.077850,0.077850,0.174078,0.080582,0.073127,...,0.174078,0.381385,0.402911,0.363636,0.667424,1.000000,0.454545,0.564076,0.363636,0.381385
48483,0.161165,0.000000,0.322329,0.219382,0.250873,0.311400,0.311400,0.348155,0.241747,0.219382,...,0.261116,0.476731,0.402911,0.454545,0.381385,0.454545,1.000000,0.483494,0.636364,0.476731
48488,0.142857,0.142857,0.071429,0.194461,0.148250,0.069007,0.069007,0.154303,0.071429,0.324102,...,0.077152,0.338062,0.285714,0.322329,0.338062,0.564076,0.483494,1.000000,0.241747,0.338062


#### Ejecutamos a partir de aca y cargamos los archivos para hacer funcionar el sistema de recomendacion

In [6]:
import pandas as pd
import numpy as np
import pyarrow
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

import requests
from bs4 import BeautifulSoup

# GUARDADO
# df_anime.to_pickle("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/df_anime_matriz_explotado.pickle")
# df_anime_matrix.to_pickle("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/df_anime_matriz_dummy.pickle")
# df_anime_similitud.to_pickle("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/anime_similitud.pickle")

# CARGA
# df_rating_complete = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/rating_complete.csv")
df_anime = pd.read_pickle ("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/df_anime_matriz_explotado.pickle")
df_anime_matrix = pd.read_pickle ("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/df_anime_matriz_dummy.pickle")
df_anime_similitud = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/anime_similitud.pickle")
base_de_datos = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/Cosas Exe/base_de_datos.csv") # Base de datos con info de animes
df_anime_base = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP 04 - Recomendacion/anime.csv") #csv BASE

In [9]:
# División del conjunto de datos en conjuntos de entrenamiento y prueba
train, test = train_test_split(df_rating_complete, random_state=42, test_size=0.2)

# Definimos una funcion para recomendar, basada en los mayores puntajes dados por Usuario y la matriz de similitud de los animes

In [40]:
# Definimos una  recomendaciones de animes similares a los ya puntuados por el usuario
def recomendame_algo (usuario=0, recomendaciones=20):

  rating = df_rating_complete[df_rating_complete["user_id"] == usuario]["rating"].max()
  if rating < 6:
    print ("parece que aún no tuviste suerte con los animes que viste, probemos cosas nuevas!")
  else:
    top_rated = df_rating_complete[(df_rating_complete["user_id"] == usuario) & (df_rating_complete["rating"] == rating)]["anime_id"].tolist()


  # Buscar animes similares
  lista_recomendaciones = []
  puntaje = []

  # iteramos a cada anime con puntuacion alta
  for anime in top_rated:
    cantidad_recomendaciones = recomendaciones
    anime_recomendado = df_anime_similitud[anime].sort_values(ascending = False)[1:cantidad_recomendaciones].index.tolist()

    # Recorremos cada anime recomendado y obtenemos su puntaje
    for recomendado in anime_recomendado:
        puntaje = df_anime_similitud[anime][recomendado]
        lista_recomendaciones.append((recomendado, puntaje))  # Agrega la tupla (anime, puntaje) a la lista

  animes_recomendados = df_anime_similitud[anime].sort_values(ascending=False).index.tolist()

  # animes_puntuados_usuario es una lista de MAL_ID de los animes puntuados por el usuario
  animes_puntuados_usuario = df_rating_complete[(df_rating_complete["user_id"] == usuario) & (df_rating_complete["rating"] != 0)]

  # Filtrar animes puntuados por el usuario de las recomendaciones
  animes_no_puntuados = [anime for anime in animes_recomendados if anime not in animes_puntuados_usuario]

  # Tomar los animes no puntuados como recomendaciones
  recomendaciones_finales = animes_no_puntuados[:cantidad_recomendaciones]

  df_resultado = base_de_datos[base_de_datos["anime_id"].isin(recomendaciones_finales)]

  # Ordenar df_resultado por la columna anime_id según el orden de recomendaciones
  df_resultado = df_resultado.sort_values(by='anime_id', key=lambda x: x.map({anime_id: i for i, anime_id in enumerate(recomendaciones_finales)})).reset_index(drop=True).head(20)

  return df_resultado

In [41]:
recomendame_algo(0)

,anime_id,name,genres,type,episodes,sypnopsis
0,2236,Toki wo Kakeru Shoujo,"Adventure, Drama, Romance, Sci-Fi",Movie,1,akoto Konno is in her last year of high school...
1,3115,Toki no Tabibito: Time Stranger,"Action, Adventure, Romance, Sci-Fi",Movie,1,Based on the novel trilogy of the same name by...
2,1116,Junkers Come Here,"Slice of Life, Drama",Movie,1,This heartwarming and unexpectedly mature stor...
3,5084,Mai Mai Shinko to Sennen no Mahou,"Adventure, Drama",Movie,1,"The story is set in 1955 in Kokuga, Hofu City,..."
4,3120,Bobby ni Kubittake,"Drama, Romance, Slice of Life",Movie,1,"""Bobby really does little else but work on his..."
5,5673,Apfelland Monogatari,"Adventure, Historical, Drama",Movie,1,pfelland is a small but beautiful country surr...
6,1032,Kumo no You ni Kaze no You ni,"Adventure, Historical, Drama, Romance",Movie,1,"Ginga, a small town girl sets out to become th..."
7,5535,Tottoi,"Adventure, Slice of Life, Kids, Drama",Movie,1,hat a sensation Tottoi felt when he discovered...
8,431,Howl no Ugoku Shiro,"Adventure, Drama, Fantasy, Romance",Movie,1,"That jumbled piece of architecture, that cacop..."
9,34744,Kumo no Gakkou,"Slice of Life, Drama",Movie,1,"Near the end of the 19th century, a young boy ..."
